# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3cf4a57ac0>
├── 'a' --> tensor([[-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323]])
└── 'x' --> <FastTreeValue 0x7f3cf4a57a00>
    └── 'c' --> tensor([[-1.4001,  0.1970, -0.5024, -0.7657],
                        [-0.3456, -0.2996, -0.2605,  1.3872],
                        [ 0.3903, -0.8025,  2.3905,  0.5628]])

In [4]:
t.a

tensor([[-0.9329, -0.4810,  0.6070],
        [ 0.1704,  1.1519,  3.1323]])

In [5]:
%timeit t.a

68.2 ns ± 0.137 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3cf4a57ac0>
├── 'a' --> tensor([[ 0.9291, -1.7234,  0.1724],
│                   [ 0.3041, -2.4371,  0.3057]])
└── 'x' --> <FastTreeValue 0x7f3cf4a57a00>
    └── 'c' --> tensor([[-1.4001,  0.1970, -0.5024, -0.7657],
                        [-0.3456, -0.2996, -0.2605,  1.3872],
                        [ 0.3903, -0.8025,  2.3905,  0.5628]])

In [7]:
%timeit t.a = new_value

70.9 ns ± 0.2 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9329, -0.4810,  0.6070],
               [ 0.1704,  1.1519,  3.1323]]),
    x: Batch(
           c: tensor([[-1.4001,  0.1970, -0.5024, -0.7657],
                      [-0.3456, -0.2996, -0.2605,  1.3872],
                      [ 0.3903, -0.8025,  2.3905,  0.5628]]),
       ),
)

In [10]:
b.a

tensor([[-0.9329, -0.4810,  0.6070],
        [ 0.1704,  1.1519,  3.1323]])

In [11]:
%timeit b.a

57.9 ns ± 0.0424 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2060,  1.4861, -0.6563],
               [-0.6341, -0.8948, -0.6944]]),
    x: Batch(
           c: tensor([[-1.4001,  0.1970, -0.5024, -0.7657],
                      [-0.3456, -0.2996, -0.2605,  1.3872],
                      [ 0.3903, -0.8025,  2.3905,  0.5628]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.133 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

834 ns ± 8.98 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 36.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 186 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 624 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3c4694cf70>
├── 'a' --> tensor([[[-0.9329, -0.4810,  0.6070],
│                    [ 0.1704,  1.1519,  3.1323]],
│           
│                   [[-0.9329, -0.4810,  0.6070],
│                    [ 0.1704,  1.1519,  3.1323]],
│           
│                   [[-0.9329, -0.4810,  0.6070],
│                    [ 0.1704,  1.1519,  3.1323]],
│           
│                   [[-0.9329, -0.4810,  0.6070],
│                    [ 0.1704,  1.1519,  3.1323]],
│           
│                   [[-0.9329, -0.4810,  0.6070],
│                    [ 0.1704,  1.1519,  3.1323]],
│           
│                   [[-0.9329, -0.4810,  0.6070],
│                    [ 0.1704,  1.1519,  3.1323]],
│           
│                   [[-0.9329, -0.4810,  0.6070],
│                    [ 0.1704,  1.1519,  3.1323]],
│           
│                   [[-0.9329, -0.4810,  0.6070],
│                    [ 0.1704,  1.1519,  3.1323]]])
└── 'x' --> <FastTreeValue 0x7f3c4694cbe0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 80.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3c3fd5a970>
├── 'a' --> tensor([[-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323],
│                   [-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323],
│                   [-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323],
│                   [-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323],
│                   [-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323],
│                   [-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323],
│                   [-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323],
│                   [-0.9329, -0.4810,  0.6070],
│                   [ 0.1704,  1.1519,  3.1323]])
└── 'x' --> <FastTreeValue 0x7f3c3fd5aee0>
    └── 'c' --> tensor([[-1.4001,  0.1970, -0.5024, -0.7657],
                        [-0.3456, -0.2996, -0.2605,  1.3872],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 130 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.6 µs ± 124 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.4001,  0.1970, -0.5024, -0.7657],
                       [-0.3456, -0.2996, -0.2605,  1.3872],
                       [ 0.3903, -0.8025,  2.3905,  0.5628]],
              
                      [[-1.4001,  0.1970, -0.5024, -0.7657],
                       [-0.3456, -0.2996, -0.2605,  1.3872],
                       [ 0.3903, -0.8025,  2.3905,  0.5628]],
              
                      [[-1.4001,  0.1970, -0.5024, -0.7657],
                       [-0.3456, -0.2996, -0.2605,  1.3872],
                       [ 0.3903, -0.8025,  2.3905,  0.5628]],
              
                      [[-1.4001,  0.1970, -0.5024, -0.7657],
                       [-0.3456, -0.2996, -0.2605,  1.3872],
                       [ 0.3903, -0.8025,  2.3905,  0.5628]],
              
                      [[-1.4001,  0.1970, -0.5024, -0.7657],
                       [-0.3456, -0.2996, -0.2605,  1.3872],
                       [ 0.3903, -0.8025,  2.3905,  0.5628]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.4001,  0.1970, -0.5024, -0.7657],
                      [-0.3456, -0.2996, -0.2605,  1.3872],
                      [ 0.3903, -0.8025,  2.3905,  0.5628],
                      [-1.4001,  0.1970, -0.5024, -0.7657],
                      [-0.3456, -0.2996, -0.2605,  1.3872],
                      [ 0.3903, -0.8025,  2.3905,  0.5628],
                      [-1.4001,  0.1970, -0.5024, -0.7657],
                      [-0.3456, -0.2996, -0.2605,  1.3872],
                      [ 0.3903, -0.8025,  2.3905,  0.5628],
                      [-1.4001,  0.1970, -0.5024, -0.7657],
                      [-0.3456, -0.2996, -0.2605,  1.3872],
                      [ 0.3903, -0.8025,  2.3905,  0.5628],
                      [-1.4001,  0.1970, -0.5024, -0.7657],
                      [-0.3456, -0.2996, -0.2605,  1.3872],
                      [ 0.3903, -0.8025,  2.3905,  0.5628],
                      [-1.4001,  0.1970, -0.5024, -0.7657],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
